# Log Analysis

In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
STYLE_TRAIN, COLOR_TRAIN = "s-", "#19c"
STYLE_EVAL, COLOR_EVAL = "^-", "#f91"

## Data Loading

In [4]:
with open("config.json", encoding="utf-8") as json_file:
    config = json.load(json_file)

In [ ]:
df_statistics = pd.read_csv("statistics.csv", index_col=0)
df_statistics.info()

In [ ]:
with open("confusion_matrices.json", encoding="utf-8") as json_file:
    confusion_matrices = json.load(json_file)

print("Count of confusion matrices:",len(confusion_matrices))

## Training Config

In [ ]:
for key, value in config.items():
    print(f"######## {key} ########")
    print(value)

## Training Time

In [ ]:
train_times_in_seconds = df_statistics["train_time_seconds"]
total_train_times_in_seconds = train_times_in_seconds.sum()
mean_train_times_in_seconds = train_times_in_seconds.mean()
std_train_times_in_seconds = train_times_in_seconds.std()

print("Total Epoch(s)  : %23d" % len(df_statistics))
print(
    "Sum(TrainTime)  = %4.0f hr %3.0f min %3.0f sec"
    % (
        total_train_times_in_seconds // 3600,
        total_train_times_in_seconds // 60,
        total_train_times_in_seconds % 60,
    )
)
print(
    "Mean(TrainTime) = %4.0f hr %3.0f min %3.0f sec"
    % (
        mean_train_times_in_seconds // 3600,
        mean_train_times_in_seconds // 60,
        mean_train_times_in_seconds % 60,
    )
)
print(
    "Std(TrainTime)  = %4.0f hr %3.0f min %3.0f sec"
    % (
        std_train_times_in_seconds // 3600,
        std_train_times_in_seconds // 60,
        std_train_times_in_seconds % 60,
    )
)

In [ ]:
fig = plt.figure(figsize=(9, 3), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.stackplot(
    df_statistics["epoch"],
    train_times_in_seconds,
    df_statistics["eval_time_seconds"],
    labels=("Train", "Eval"),
    colors=(COLOR_TRAIN, COLOR_EVAL),
    alpha=0.2,
)
ax.plot(
    df_statistics["epoch"],
    train_times_in_seconds,
    STYLE_TRAIN,
    color=COLOR_TRAIN,
)
ax.plot(
    df_statistics["epoch"],
    train_times_in_seconds + df_statistics["eval_time_seconds"],
    STYLE_EVAL,
    color=COLOR_EVAL,
)
ax.set(
    title="Epoch Times",
    xlabel="Epoch",
    ylabel="Seconds",
    xticks=np.arange(
        df_statistics["epoch"].min(),
        df_statistics["epoch"].max() + 0.1,
        np.ceil(len(df_statistics) / 10),
    ),
)
ax.legend()
ax.grid(axis="y")

## Accuracy & Loss

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150)
fig.set_facecolor("#fff")
ax_acc = fig.add_subplot()
ax_loss = ax_acc.twinx()

eval_mask = ~df_statistics["eval_accuracy"].isna()

ax_loss.legend(
    handles=[
        bars := ax_acc.bar(
            df_statistics["epoch"][eval_mask],
            df_statistics["eval_accuracy"][eval_mask],
            color="lightgreen",
            alpha=0.5,
            label="eval_accuracy",
        ),
        *ax_loss.plot(
            df_statistics["epoch"],
            df_statistics["train_avg_loss"],
            STYLE_TRAIN,
            color=COLOR_TRAIN,
            label="train_avg_loss",
        ),
        *ax_loss.plot(
            df_statistics["epoch"][eval_mask],
            df_statistics["eval_avg_loss"][eval_mask],
            STYLE_EVAL,
            color=COLOR_EVAL,
            label="eval_avg_loss",
        ),
    ],
    loc="center right",
)

for bar in bars:
    bar_height = bar.get_height()
    ax_acc.text(
        bar.get_x() + bar.get_width() / 2,
        bar_height,
        f"{bar_height:.3f}",
        ha='center',
        va='bottom'
    )

ax_acc.set(
    ylabel="Accuracy",
    title="Model Statistics",
    xlabel="Epoch",
    xticks=df_statistics["epoch"],
)
ax_loss.set(
    ylabel="Loss",
)
ax_acc.grid(axis="y", alpha=0.3)

## Confusion Matrix

In [ ]:
last_confusion_matrix = np.array(confusion_matrices[-1]["confusion_matrix"])
last_relative_confusion_matrix = (
    last_confusion_matrix / last_confusion_matrix.sum(axis=1)
)

fig = plt.figure(figsize=(8, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

sns.heatmap(
    last_relative_confusion_matrix,
    fmt=".2f",
    annot=True,
    cmap="coolwarm",
)
ax.set(
    title="Relative Confusion Matrix",
    xlabel="Prediction",
    ylabel="Ground Truth",
)

last_confusion_matrix